In [226]:
import pandas as pd
import numpy as np
from collections import Counter
import tensorflow as tf
import datetime
from gensim.models import Word2Vec, KeyedVectors
from konlpy.tag import Okt
import random
import copy
from sklearn.model_selection import train_test_split

In [2]:
DF = pd.read_csv('labeled_df.csv')

# 사전학습된 벡터 로드하기

In [3]:
# pretrained_vector
snu_vector = KeyedVectors.load_word2vec_format('snu_news_word2vec.vec',binary=False)

snu_vocab = snu_vector.vocab

# 단어별 카운팅을 세봤는데 이상하게 되어있음..?
w2c = dict()
for item in snu_vocab:
    w2c[item]=snu_vocab[item].count

In [4]:
print("사전학습된 워드벡터의 차원\n",np.shape(snu_vector[list(snu_vocab)[0]]))
# 200차원으로 훈련시켰으니까 내벡터도 200차원으로 훈련시키자.

사전학습된 워드벡터의 차원
 (200,)


# 리뷰를 문자열로 치환하기

In [5]:
# 데이터 손실을 방지하기 위해서 문자열로 저장하고싶음.
# 리뷰 -> 트위터 형태소분석기 -> word2vec 객체의 인덱스로 변환 -> 리스트를 문자열로 병합 -> 저장

# 위의 pretrained vector도 트위터 형태소분석기를 사용한것으로 보여짐
# 이것도 트위터 형태소분석기 따라서 분해한다음에 워드벡터를 만들어보자.
okt = Okt()

def basic_preprocessing(string):
    if string != string or type(string) == type(1) or string == ' ':
        return ''
    else:
        return string
DF['교정된 리뷰'] = DF['교정된 리뷰'].apply(basic_preprocessing)

def twitter(string):
    tagged_list = okt.pos(string) 
    
    if random.uniform(0,1) < 1e-3 and len(tagged_list) >= 5:
        print('--------------------')
        print('교정전 리뷰\n',string)
        print('\n교정후 리뷰\n','/'.join([x for x,y in tagged_list]))
        
    return tagged_list

tagged_review = DF['교정된 리뷰'].apply(twitter)

--------------------
교정전 리뷰
 초보들이 하기에 쉽게 하는게 좋을것같아요

교정후 리뷰
 초보/들/이/하기에/쉽게/하는게/좋을것/같아요
--------------------
교정전 리뷰
 이모티콘   !!   빨리받음좋겄다   ㅋㅋ  

교정후 리뷰
 이모티콘/!!/빨리/받음/좋/겄/다/ㅋㅋ
--------------------
교정전 리뷰
 친구 추천으로 하는데 재밌음

교정후 리뷰
 친구/추천/으로/하는데/재밌음
--------------------
교정전 리뷰
 킬링타임용으로 할만하네요

교정후 리뷰
 킬링타임/용/으로/할만/하네요
--------------------
교정전 리뷰
  Num    인데 잘들어가지다가 왜 갑자기 새벽부터 안들어가는지    Elip     답답 

교정후 리뷰
 Num/인데/잘/들어가지다가/왜/갑자기/새벽/부터/안/들어가는지/Elip/답답
--------------------
교정전 리뷰
 게임은 재밌는데 현질이 좀 심하네요    Elip    살짝만 풀어주심 좋을거같아요

교정후 리뷰
 게임/은/재밌는데/현질/이/좀/심하네요/Elip/살짝/만/풀어/주심/좋을거/같아요
--------------------
교정전 리뷰
 완전 재밌었요    Elip   !!   그리고 실제로 자기가 키운 농작물을 , 배송해주니까 더좋은같아요 !  ^^  

교정후 리뷰
 완전/재밌었요/Elip/!!/그리고/실제/로/자기/가/키운/농작물/을/,/배송/해주니까/더/좋은/같아요/!/^^
--------------------
교정전 리뷰
 아니 처음 들어갈때 왜 통신오류 와이파이  빵빵 한데

교정후 리뷰
 아니/처음/들어갈/때/왜/통신/오류/와이파이/빵빵/한데
--------------------
교정전 리뷰
 작물 시들지 않게좀    Elip    어떻게 안될까영

교정후 리뷰
 작물/시들지/않게/좀/Elip/어떻게/안될까/영
--------------------
교정전 리뷰
 오늘 새벽    Num    시부터 접속이 안되네

--------------------
교정전 리뷰
 우리정음이언니가살렷다

교정후 리뷰
 우리/정음/이/언니/가/살렷다
--------------------
교정전 리뷰
 아니 뭐하는거죠 ? 자동사냥 돌렸는데 다야가 사라져있어요 자사할때 자동부활도 꺼져있고 지금 부활석도 남아있고 다야 왜 날라간거죠 ?

교정후 리뷰
 아니/뭐/하는거죠/?/자동사/냥/돌렸는데/다야/가/사라져있어요/자사/할/때/자동/부활/도/꺼져있고/지금/부활/석도/남아있고/다야/왜/날라/간거죠/?
--------------------
교정전 리뷰
 아니 현질을 해서 다이아를 얻어도 육성에도움되는걸못사고 상점도 이상한것들만 있고 또한 경매장있는건좋지만 육성에는 크게 도움안되고 또한 탈것    펫    이벤트뽑기등을사도 너무육성에도움되는게 안뜨고 재료만뜨니깐 짜증나요 그리고 다이아현질한다해서 강해지는방법도 별로없고 좀 전체적으로 패치좀 부탁드려요 그리고 쿠폰번호좀알려주세요

교정후 리뷰
 아니/현질/을/해서/다이아/를/얻어도/육성/에/도움/되는걸/못/사고/상점/도/이상한것/들만/있고/또한/경/매장/있는건/좋지만/육성/에는/크게/도움/안되고/또한/탈것/펫/이벤트/뽑기/등/을/사도/너무/육성/에/도움/되는게/안/뜨고/재료/만/뜨니깐/짜증나요/그리고/다이아/현질/한다해/서/강해지는/방법/도/별로/없고/좀/전체/적/으로/패치/좀/부탁드려요/그리고/쿠폰/번호/좀/알려주세요
--------------------
교정전 리뷰
 제가 오랜만에 들어왔는데 캐릭터가 없어졌어요 문의는 했는데 빨리 처리좀

교정후 리뷰
 제/가/오랜/만/에/들어왔는데/캐릭터/가/없어졌어요/문의/는/했는데/빨리/처리/좀
--------------------
교정전 리뷰
  굳굳  재밋습니다   !!  

교정후 리뷰
 굳/굳/재밋습니/다/!!
--------------------
교정전 리뷰
 재밌어요 인생갬이요 그리고 없는공룡알려드릴게요 업데이트 안하셔도 되요    Num    코엘로피시스 육    Num     리리 엔스테르누스 육

--------------------
교정전 리뷰
 재미있게 잘하고 있습니다    Elip 

교정후 리뷰
 재미있게/잘/하고/있습니다/Elip
--------------------
교정전 리뷰
 아니 해보지도 못했어요   !!   왜    Num    퍼만 오면 네트워크 연결 안된다고 떠요 ?  !!  

교정후 리뷰
 아니/해보지도/못/했어요/!!/왜/Num/퍼/만/오면/네트워크/연결/안된다고/떠요/?/!!
--------------------
교정전 리뷰
 굿겜 ~ 시간때우기 딱좋은게임이군요

교정후 리뷰
 굿겜/~/시간/때우기/딱좋은/게임/이군/요
--------------------
교정전 리뷰
 다좋은데요 캐릭터 확률좀 올려주셈요 연속으로 프리미엄    Num    개    Num    번깠는데    Num    성    Num    개나왔어요

교정후 리뷰
 다/좋은데/요/캐릭터/확률/좀/올려주/셈/요/연속/으로/프리미엄/Num/개/Num/번/깠는데/Num/성/Num/개/나왔어요
--------------------
교정전 리뷰
 게임은 들어가는데 모험은 로딩이 넘 길어서 안들어 가지네요

교정후 리뷰
 게임/은/들어가는데/모험/은/로딩/이/넘/길어서/안/들어/가지네요
--------------------
교정전 리뷰
 성장시키기힘들고 캐릭터가 다양히 안나옮니다

교정후 리뷰
 성장/시키기/힘들고/캐릭터/가/다양히/안나/옮니/다
--------------------
교정전 리뷰
 우왕 이런 해전게임 처음이에요   !!   어쌔신크리드 배타는 그런것보다 훨씬 퀄리티 좋고 게임 진행 도도  훨씬만족   !!  ( 요즘 오버워치  슬슬 질렸는데 잘됬다   !!  )

교정후 리뷰
 우왕/이런/해전/게임/처음/이에요/!!/어쌔신크리드/배타/는/그런/것/보다/훨씬/퀄리티/좋고/게임/진행/도도/훨씬/만족/!!/(/요즘/오버/워치/슬슬/질렸는데/잘/됬다/!!/)
--------------------
교정전 리뷰
 스토리가 좋아요몰입도최고네요   !!  

교정

--------------------
교정전 리뷰
 다좋은데    Elip    게임중    Elip    적이 처치되엇 습니다    이런거 말고 영어로 고쳐주면 안되나여 ? 그리고    Num    인 입장 추가 부탁드리구여    Elip    새로 추가한 월    Num    원짜리 헤택이 넘 적어여    Elip    머거이 쓸모 없는거 같음    Elip 

교정후 리뷰
 다/좋은데/Elip/게임/중/Elip/적/이/처치/되/엇/습니다/이런거/말고/영어/로/고쳐주면/안되나여/?/그리고/Num/인/입장/추가/부탁드리구여/Elip/새로/추가/한/월/Num/원/짜리/헤택/이/넘/적어여/Elip/머거이/쓸모/없는거/같음/Elip
--------------------
교정전 리뷰
 다좋은데 핵만좀 얼른잡아주세요

교정후 리뷰
 다/좋은데/핵/만/좀/얼른/잡아주세요
--------------------
교정전 리뷰
 잼있어요 다이아도 많이줘요 강추합니다

교정후 리뷰
 잼있어요/다이아/도/많이줘요/강/추합니다
--------------------
교정전 리뷰
 오늘은 슬픈 날입니다    제가 처음으로 이벤트를 못 끝낸 날이기 때문이죠    현재 기준으로    Num    시간 남았습니다만 하얀 니토에서 막히고 현재 닥돌 할 시간도 안되서 포기합니다    왜 이벤트는 항상 시험기간에 하는 건가요 ?

교정후 리뷰
 오늘/은/슬픈/날/입니다/제/가/처음/으로/이벤트/를/못/끝낸/날/이기/때문/이/죠/현재/기준/으로/Num/시간/남았습니다만/하얀/니/토/에서/막히고/현재/닥돌/할/시간/도/안되서/포기/합니다/왜/이벤트/는/항상/시험/기간/에/하는/건가/요/?
--------------------
교정전 리뷰
 아니 무슨 이벤트전역 하러가면 도르마무냐 뭘 해먹을수가 없고 아직도 그냥 중장비 편집으로 살펴보는대도 팅기고 조취도 똑바로 안되고 보급상자 딸랑 던져주고 끝인가 진짜 게임시작하고 반년간 즐겁게 했는데 근래 운영은 진짜 최악중에 악이다

교정후 리뷰
 아니/무슨/이

--------------------
교정전 리뷰
 빛암 소환서 번개쳐라 얍   !!  

교정후 리뷰
 빛암/소환/서/번개/쳐라/얍/!!
--------------------
교정전 리뷰
 시간 가는줄 모르겠다 진짜 강추   !!  

교정후 리뷰
 시간/가는줄/모르겠다/진짜/강추/!!
--------------------
교정전 리뷰
 자꾸 게임이 갑작스레 시작이 안된다그러네요 ? 뭐 어떡하라는거죠

교정후 리뷰
 자꾸/게임/이/갑작스레/시작/이/안된다/그러네요/?/뭐/어떡하라는/거/죠
--------------------
교정전 리뷰
 자꾸 튕겨요 빨리 고쳐주세요 현기증 난단 말이야 블리자드 믿는다

교정후 리뷰
 자꾸/튕겨/요/빨리/고쳐주세요/현기증/난단/말/이야/블리자드/믿는다
--------------------
교정전 리뷰
 이번 추석때 친척들이랑 포커 쳤는데    Elip    많이 도움 됐어요   !!  

교정후 리뷰
 이번/추석/때/친척/들/이랑/포커/쳤는데/Elip/많이/도움/됐어요/!!
--------------------
교정전 리뷰
 출근길이 가까워 가볍게 한두판 즐기며 하루를 시작 합니다    

교정후 리뷰
 출근길/이/가까워/가볍게/한두/판/즐기며/하루/를/시작/합니다
--------------------
교정전 리뷰
 사람이 없어 누구랑치냐 ㅋ

교정후 리뷰
 사람/이/없어/누구/랑/치냐/ㅋ
--------------------
교정전 리뷰
 너무하네 캐쉬충전도했는데 접속도 안됨   ㅡㅡ   와이파이켰는데도 무슨 사기꾼들아니냐

교정후 리뷰
 너무하네/캐쉬/충전/도/했는데/접속/도/안됨/ㅡㅡ/와이파이/켰는데도/무슨/사기꾼/들아니냐
--------------------
교정전 리뷰
 좋은데 이닝당    Num ~ Num    번 몸 맞는 공이 생기면 벤치클리어닝이 되조록 해 주시면 감사하겠습니다 ~ 그리고 요류가 있는게 아래 분처럼 게임이 자동으로 들어가져서 충전기 꽂고 아침에 보는데    Num % 밖애 충전이 안되는    E

In [6]:
# pretrained와 같은형태로 사용하기 위해서 이런식으로 바꿈.
tagged_review_joined = [["('%s','%s')"%(x,y) for x,y in review] for review in tagged_review]

In [7]:
review_model = Word2Vec(tagged_review_joined, size=200, window = 5, workers=6, iter=10, sg=1)

In [97]:
review_vector = review_model.wv
review_vocab = review_vector.vocab

# 카운팅 객체(나중에 쓸거임)
w2c_review = dict()
for item in sorted(review_vocab,key=review_vocab.get,reverse=True):
    w2c_review[item]=review_vocab[item].count

In [98]:
w2c_review

{"('이','Josa')": 88429,
 "('Num','Alpha')": 83979,
 "('게임','Noun')": 80791,
 "('Elip','Alpha')": 67854,
 "('가','Josa')": 58848,
 "('에','Josa')": 48035,
 "('을','Josa')": 46384,
 "('도','Josa')": 44731,
 "('?','Punctuation')": 38479,
 "('너무','Adverb')": 29804,
 "('은','Josa')": 27736,
 "('좀','Noun')": 25357,
 "('안','VerbPrefix')": 25223,
 "('이','Determiner')": 25108,
 "('요','Josa')": 24796,
 "('는','Josa')": 23705,
 "('를','Josa')": 23189,
 "('들','Suffix')": 21469,
 "('다','Adverb')": 19120,
 "('로','Josa')": 18968,
 "('만','Josa')": 18921,
 "('으로','Josa')": 18917,
 "('!','Punctuation')": 18878,
 "('굿','Noun')": 17402,
 "('겜','Noun')": 17297,
 "('좋아요','Adjective')": 16179,
 "('왜','Noun')": 15784,
 "('거','Noun')": 14913,
 "('!!','Punctuation')": 14439,
 "('시간','Noun')": 14314,
 "('의','Josa')": 13956,
 "('에서','Josa')": 13925,
 "('다시','Noun')": 13841,
 "('~','Punctuation')": 13647,
 "('입니다','Adjective')": 13643,
 "(',','Punctuation')": 13217,
 "('할','Verb')": 13121,
 "('진짜','Noun')": 12945,
 "('때'

In [10]:
word_dict = {x:y for y,x in enumerate(review_vocab.keys())}
temp1 = word_dict["('UNK','Alpha')"] 
word_dict["('좋음','Adjective')"] = temp1
word_dict["('UNK','Alpha')"] = 0

inv_dict = list(word_dict.keys())
inv_dict[0], inv_dict[temp1] = inv_dict[temp1], inv_dict[0]

In [12]:
def word_to_index(word):
    try:
        return word_dict[word]
    except:
        return 0

indexed_review = [[word_to_index(word) for word in review] for review in tagged_review_joined]

# data 손실을 방지하기 위해서 문자열로 합쳐서 보내자.
indexed_review_string = ['/'.join([str(x) for x in e]) for e in indexed_review]

DF['문자열'] = indexed_review_string

In [14]:
print("리뷰 하나만 프린트해보기\n----------------------")
print(*[DF['교정된 리뷰'].values[115],tagged_review[115],indexed_review[115],indexed_review_string[115]],sep='\n\n')

리뷰 하나만 프린트해보기
----------------------
친구가 소개해주어서 해보았는데 너무 재미있네요  짱짱    😀😀    👍👍  

[('친구', 'Noun'), ('가', 'Josa'), ('소개', 'Noun'), ('해주어서', 'Verb'), ('해보았는데', 'Verb'), ('너무', 'Adverb'), ('재미있네요', 'Adjective'), ('짱짱', 'Noun'), ('\uf600\uf600', 'Foreign'), ('\uf44d\uf44d', 'Foreign')]

[138, 49, 363, 364, 365, 53, 366, 84, 367, 368]

138/49/363/364/365/53/366/84/367/368


# 사전학습된 벡터와 비교해보기

In [28]:
def get_lookup(vectors,inv_dict):
    dim = 200
    vocab_size = len(inv_dict)
    print(dim,vocab_size)
    
    lookup_matrix = np.zeros((vocab_size,dim))
    for i in range(vocab_size):
        try:
            lookup_matrix[i] = vectors[inv_dict[i]]
        except:
            pass
        
    return lookup_matrix


review_lookup = get_lookup(review_vector,inv_dict)
pretr_lookup = get_lookup(snu_vector,inv_dict)

200 29782
200 29782


In [76]:
# 38%의 단어만 겹칠뿐이었다고한다..
intersection_with_zeros = np.equal(pretr_lookup,np.zeros((29782,200))).astype(int)

print("사전학습 벡터에 없는 단어의 비율\n",
      np.mean(intersection_with_zeros))

사전학습 벡터에 없는 단어의 비율
 0.6231280639312337


In [83]:
lookup = np.zeros((29782,200))

# 사전학습벡터와 우리의 벡터 합치기
for x in range(29782):
    if intersection_with_zeros[x][0] == 0:
        lookup[x] = 0.5*(review_lookup[x] + pretr_lookup[x])
    else:
        lookup[x] = review_lookup[x]

# 학습 데이터 만들기

In [26]:
"""
일단은 평점기준으로 클러스터링을 할 예정임.

1. 리뷰를 인덱스로 바꾼다
2. 인덱스로 바꾼 리뷰를 데이터프레임에 넣는다
3. 데이터프레임에 넣고, 게임별로 리뷰를 분리한다.
4. 
    1) 각 게임별 리뷰
        - 일단 평점별로 분리한다.
        - 평점별로 분리해서 각 데이터프레임당 100개의 글을 랜덤하게 20번 뽑는다.
        - 다시 합쳐서 20개의 샘플을 만든다.
        - 각 샘플당 카운터 객체를 만든다.
    2) tf-idf 가중치
        - 일단 idf를 구하기 위해서 w2c_review의 카운팅 개수를 가져온다.
        - 1)에서 만든 카운터객체와 비교해서 tf-idf 가중치를 뽑아낸다.
        - tf-idf 가중평균을 통해서 document vector를 만든다.
    3) 테스트
        - 샘플을 만들고 
"""

COPY = DF[['평점','문자열','이름','is_good']]

GROUP_BY_GAME = COPY.groupby(['이름'])

keys = list(GROUP_BY_GAME.groups.keys())
indices = list(GROUP_BY_GAME.groups.values())

DFs = [COPY.loc[indices[x]] for x in range(len(indices))]

In [144]:
# idf 숫자 만들기

total_count_sum = sum(w2c_review.values())

idf_score = {x:np.log(total_count_sum/w2c_review[inv_dict[x]]) for x in range(len(w2c_review))}

idf_score

{0: 6.733636691182925,
 1: 6.54329299157489,
 2: 3.9741672164156485,
 3: 7.135816491037435,
 4: 9.522529092088636,
 5: 4.532804072142051,
 6: 12.09361343811769,
 7: 13.233047721306054,
 8: 11.574819644702522,
 9: 5.261055955701455,
 10: 6.067264489156205,
 11: 8.349299277115652,
 12: 6.273055842988408,
 13: 11.063994020936532,
 14: 11.786128738369728,
 15: 5.823230104228084,
 16: 7.9498439925680655,
 17: 4.604067210448512,
 18: 9.914326561468137,
 19: 7.715092942773095,
 20: 6.652581969370478,
 21: 11.23495181908017,
 22: 5.88873249545408,
 23: 6.98355522103036,
 24: 9.393595408712743,
 25: 9.262755807753932,
 26: 8.687096894977934,
 27: 3.9225340138595586,
 28: 6.314599812785174,
 29: 6.037391643794429,
 30: 11.911291881323734,
 31: 6.058706333166871,
 32: 7.204468041609143,
 33: 5.777098782089188,
 34: 6.9559344175324656,
 35: 7.208390206842308,
 36: 8.385912229763164,
 37: 6.521155269684539,
 38: 9.401692618945363,
 39: 6.450855665299263,
 40: 5.734801105014671,
 41: 9.9098118811136

In [145]:
def counter_to_vector(counter):
    # tf : log scale frequency를 사용하겠다.
    tf_idf = {int(x):idf_score[int(x)]*np.log(y+1) for x,y in counter.items()}
    unique_word = len(tf_idf)
    weighted_vector = np.zeros(200)
    for idx,weight in tf_idf.items():
        weighted_vector += weight*lookup[idx]
    weighted_vector /= unique_word
    return weighted_vector

In [211]:
data = []

for DF in DFs:
    indices_by_score = DF.groupby(['평점']).groups
    label = DF['is_good'].values[0]
    game_name = DF['이름'].values[0]
    print(game_name,':',label)

    # 전체 리뷰를 일단 수집하자.
    review_by_score = ['/'.join(DF.loc[indices_by_score[x]]['문자열']).split('/') for x in range(1,6)]
    review_by_score = [[y for y in review if y != ''] for review in review_by_score]
    counter_by_score = [Counter(review) for review in review_by_score]
    game_vector = np.concatenate([counter_to_vector(cn) for cn in counter_by_score])

    # 100개의 샘플을 수집하자. (without replacement)
    sample_vectors = np.zeros((100,1000))
    for idx in range(100):
        sample_vector = np.zeros(1000)
        for x in range(1,6):
            if len(indices_by_score[x]) >= 100:
                sample_string = DF.loc[np.random.choice(indices_by_score[x],100,replace=False)]['문자열']
            else:
                sample_string = DF.loc[indices_by_score[x]]['문자열']
            sample_indices = [x for x in '/'.join(sample_string).split('/') if x != '']
            counter = Counter(sample_indices)
            sample_vector[200*(x-1):200*x] = counter_to_vector(counter)

        sample_vectors[idx] = sample_vector
        
    data.append([label,game_vector,sample_vectors])

2018갓오브하이스쿨 with NAVER WEBTOON : False
FINAL FANTASY BRAVE EXVIUS : False
Idle Heroes - 아이들 히어로즈 : False
Mobile Strike : False
Sdorica - sunset - (스도리카 -선셋-) : False
War Machines: 탱크 게임 - 무료 : False
Yu-Gi-Oh! Duel Links : False
 Jurassic World™: The Game : False
검과마법 for Kakao : False
권력:THERULERS : False
낚시의 신 : False
냥코 대전쟁 : False
넷마블 포커 - 바카라, 7포커, 로우바둑이, 뉴포커 : True
노블레스 with NAVER WEBTOON : False
놀러와 마이홈 : False
다크어벤저3 : False
더 킹 오브 파이터즈98 UM온라인 for kakao : False
도미네이션즈 : False
라스트 쉘터 (Last Shelter: Survival) : True
라스트 엠파이어 워 Z : False
레알팜 : False
레이븐: SIGN : False
로드 모바일: 제국의 전쟁 - MMORPG : True
리니지2 레볼루션 : True
리니지M : True
마스터 오브 이터니티 : False
마피아 시티 : True
매직 러쉬:히어로즈(Magic Rush: Heroes) : False
메이플스토리M : False
모두의마블 for kakao : True
몬스터 길들이기 for kakao : False
몬스터슈퍼리그 : False
뮤오리진 : False
반지 : False
별이되어라! : False
사커스피리츠 : False
삼국블레이드 : False
삼국지 조조전 Online : False
섀도우버스(Shadowverse) : False
서머너즈 워: 천공의 아레나 : True
세븐나이츠 for Kakao : True
소녀전선 : False
소드 아트 온라인　-메모리 디프래그- : False

# 모델 만들기

In [310]:
"""
1. 게임 단위로 테스트셋과 트레인셋을 분리하기.
2. 테스트셋,트레인셋에서 sample_vectors들과 라벨을 전부 가져와서 합친 다음에 다시 셔플
3. 모델을 잘 학습한 다음에 진짜 game_vector들에 대해서도 잘맞히는지 테스트해보기.
"""

# 80: 16의 비율로 적절히 분리해보자.
# 데이터 보존을 위해 deepcopy를 이용해서 깊은 복사를 해주자.
DATA = copy.deepcopy(data)
random.shuffle(DATA)
train_set, test_set = DATA[:80],DATA[80:]

In [311]:
train_vector = np.zeros((8000,1000))
train_label = np.zeros((8000,2))
for x in range(80):
    label, game_vector, sample_vectors = train_set[x]
    train_vector[x*100:(x+1)*100] = sample_vectors
    labels = np.zeros((100,2))
    if label == True:
        labels[:,0] = 1
        train_label[x*100:(x+1)*100] = labels
    else:
        labels[:,1] = 1
        train_label[x*100:(x+1)*100] = labels
        
X_train, _, y_train, _ = train_test_split(train_vector,train_label,test_size=0,random_state=50)

In [312]:
test_vector = np.zeros((1600,1000))
test_label = np.zeros((1600,2))
for x in range(16):
    label, game_vector, sample_vectors = test_set[x]
    test_vector[x*100:(x+1)*100] = sample_vectors
    labels = np.zeros((100,2))
    if label == True:
        labels[:,0] = 1
        test_label[x*100:(x+1)*100] = labels
    else:
        labels[:,1] = 1
        test_label[x*100:(x+1)*100] = labels
        
X_test, _, y_test, _ = train_test_split(test_vector,test_label,test_size=0,random_state=50)

In [313]:
print("shape of train data\n",np.shape(X_train),np.shape(y_train))
print("shape of test data\n",np.shape(X_test),np.shape(y_test))

shape of train data
 (8000, 1000) (8000, 2)
shape of test data
 (1600, 1000) (1600, 2)


In [326]:
def train(X_train,X_test,Y_train,Y_test,learning_rate=1e-2):
    tf.reset_default_graph()
    train_size, emb_size = np.shape(X_train)
    test_size, _ = np.shape(X_test)
        
    with tf.Graph().as_default():
        config = tf.ConfigProto(
            log_device_placement=False,
            allow_soft_placement=True
        )
        config.gpu_options.allocator_type = 'BFC'
        sess = tf.Session(config=config)
        with sess.as_default():
            
            initializer = tf.contrib.layers.xavier_initializer()
            X = tf.placeholder(tf.float32, shape=[None,emb_size], name='input')
            Y = tf.placeholder(tf.float32, shape=[None,2], name='label')
            batch_size = 32
            dropout_rate = tf.placeholder("float")

            with tf.device('/gpu:0'), tf.name_scope("fully-connected"):
            
                W_fc1 = tf.get_variable("W_fc1", shape=[1000,100], initializer=initializer)
                b_fc1 = tf.get_variable("b_fc1", shape=[100], initializer=initializer)
                fc_ = tf.nn.relu(tf.matmul(X,W_fc1)+b_fc1, name="fc")
                fc  = tf.nn.dropout(fc_,dropout_rate)
                
                W_fc2 = tf.get_variable("W_fc2", shape=[100,2], initializer=initializer)
                b_fc2 = tf.get_variable("b_fc2", shape=[2], initializer=initializer)
                out = tf.nn.xw_plus_b(fc,W_fc2,b_fc2)
            
            with tf.device('/gpu:0'), tf.name_scope("prediction"):
                
                costs = tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=Y)
                cost = tf.reduce_mean(costs)
                global_step = tf.Variable(0, name="global_step", trainable=False)
                train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost,global_step=global_step)
                prob = tf.nn.softmax(out)
                prediction = tf.argmax(prob,axis=1)
                label = tf.argmax(Y,axis=1)
                is_equal = tf.equal(prediction,label)
                accuracy = tf.reduce_mean(tf.cast(is_equal,tf.float32))
                
            total_loop = int(train_size/batch_size)
            print("total_loop:",total_loop)
            sess.run(tf.global_variables_initializer())

            for epoch in range(5):
                for idx in range(total_loop):

                    _,c,a,lb,pr = sess.run([train_op,cost,accuracy,label,prediction],
                                     {X:X_train[batch_size*idx:batch_size*(idx+1)],
                                      Y:Y_train[batch_size*idx:batch_size*(idx+1)],
                                     dropout_rate:0.6})

                    if idx == epoch:
                        print("epoch = %s, step = %s, train loss = %s, train accuracy = %s"%(epoch,idx,round(c,2),round(a,2)))                    
                    
                    if idx % 50 == 0 and idx != 0:
                        print("epoch = %s, step = %s, train loss = %s, train accuracy = %s"%(epoch,idx,round(c,2),round(a,2)))
                    

                print('\nEvaluation')

                feed_dict_test = {
                X : X_test,
                Y : Y_test,
                dropout_rate:1
                }

                confusion = tf.confusion_matrix(label,prediction)
                
                cost_t, acc_t,matrix = sess.run(
                [cost, accuracy,confusion], feed_dict = feed_dict_test)

                time_str = datetime.datetime.now().isoformat()
                print(time_str,'\n')
                print("epoch = %s, step = %s, test loss = %s, test accuracy = %s"%(epoch,idx,round(cost_t,2)
                                                                                   ,round(acc_t,2)))
                
                print("confusion_matrix\n",matrix)
                
                print('\n')


In [327]:
# 영 만족스럽지 않다...

train(X_train,X_test,y_train,y_test,5e-4)

total_loop: 250
epoch = 0, step = 0, train loss = 1.05, train accuracy = 0.28
epoch = 0, step = 50, train loss = 0.54, train accuracy = 0.72
epoch = 0, step = 100, train loss = 0.42, train accuracy = 0.91
epoch = 0, step = 150, train loss = 0.29, train accuracy = 0.91
epoch = 0, step = 200, train loss = 0.41, train accuracy = 0.81

Evaluation
2018-12-07T20:01:16.402622 

epoch = 0, step = 249, test loss = 0.55, test accuracy = 0.74
confusion_matrix
 [[   0  400]
 [  21 1179]]


epoch = 1, step = 1, train loss = 0.36, train accuracy = 0.88
epoch = 1, step = 50, train loss = 0.38, train accuracy = 0.84
epoch = 1, step = 100, train loss = 0.28, train accuracy = 0.94
epoch = 1, step = 150, train loss = 0.21, train accuracy = 0.94
epoch = 1, step = 200, train loss = 0.3, train accuracy = 0.94

Evaluation
2018-12-07T20:01:16.670171 

epoch = 1, step = 249, test loss = 0.57, test accuracy = 0.7
confusion_matrix
 [[  19  381]
 [  99 1101]]


epoch = 2, step = 2, train loss = 0.27, train accura